# Semantic search with FAISS (TensorFlow)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece] -q
!pip install faiss-gpu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 2.1 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

issues_dataset = load_dataset("lewtun/github-issues", split="train")
issues_dataset

Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 3019
})

In [3]:
issues_dataset = issues_dataset.filter(
    lambda x: (x["is_pull_request"] == False and len(x["comments"]) > 0)
)
issues_dataset

Filter:   0%|          | 0/3019 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 808
})

In [4]:
columns = issues_dataset.column_names
columns_to_keep = ["title", "body", "html_url", "comments"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
issues_dataset = issues_dataset.remove_columns(columns_to_remove)
issues_dataset

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 808
})

In [5]:
issues_dataset.set_format("pandas")
df = issues_dataset[:]

In [6]:
df["comments"][0].tolist()

['Cool, I think we can do both :)',
 '@lhoestq now the 2 are implemented.\r\n\r\nPlease note that for the the second protection, finally I have chosen to protect the master branch only from **merge commits** (see update comment above), so no need to disable/re-enable the protection on each release (direct commits, different from merge commits, can be pushed to the remote master branch; and eventually reverted without messing up the repo history).']

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df= pd.read_csv('/content/drive/MyDrive/semanticsearchPython/tenderdataforsearch.csv')

<ipython-input-3-6eb9126d7a0e>:2: DtypeWarning: Columns (0,9,12,17,19,21,24,27,29,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('/content/drive/MyDrive/semanticsearchPython/tenderdataforsearch.csv')


In [4]:
df

,Unnamed: 0,address,cityname,closingdate,companyname,companysubindustryname,continent,countryname,currcode,currencyid,...,subindustryname,tcno,tenderdate,tenderid,tenderprocid,tenderrefno,tendersbrief,tendersbriefnew,tendervalue,uploadeddate
0,0,indianoil||refineries||panipat refinery||contr...,panipat,2022-08-10T00:00:00,indian oil corporation limited,crude oil / natural gas / mineral fuels,asia,india,rs.,174,...,machine tools,51274194,2022-08-09T00:00:00,0.0,53520888,2022_PR_153586_1,NaN,"Supply, erection and commissioning of new inle...",0.0,2023-06-06T11:22:57.890000
1,1,md (drinking water) peyjal nigam gouk||ce - ku...,udham singh nagar,2022-08-24T00:00:00,uttaranchal payjal nigam,corporations/ assoc/ chambers/ govt agencies,asia,india,inr,28,...,water storage and supply,52175715,2022-08-19T00:00:00,0.0,54421897,2022_MDPEY_47852_1,NaN,Narayanpur kotha peyjal yojna in vikas khand r...,0.0,2023-06-06T11:22:57.890000
2,2,"taman duta, 50480 kuala lumpur, federal territ...",not classified,2022-08-22T00:00:00,parlimen malaysia,non classified,asia,malaysia,myr,100,...,switchgear/switching apparatus,52169972,2022-08-13T00:00:00,0.0,54416156,PAR.2/367 SH.33/2022/A,NaN,Calibration Works (ujitath) And Testing Of Rel...,0.0,2023-06-06T11:22:57.890000
3,3,"russian federation, 684000, kamchatka territor...",not classified,2022-08-23T00:00:00,state budgetary institution of health of the k...,non classified,europe,russia,rub,118,...,health services/equipments,52236294,2022-08-18T00:00:00,0.0,54482429,0338300047922000233,NaN,The Supply Of Reagents And Consumables For Hem...,2473284.0,2023-06-06T11:22:57.890000
4,4,"russian federation, 600000, vladimir region, v...",not classified,2022-08-24T00:00:00,department of property and land relations of t...,non classified,europe,russia,rub,118,...,drugs and pharmaceuticals,52184854,2022-08-15T00:00:00,0.0,54431039,0128200000122004602,NaN,The Provision Of Services For Providing Drugs ...,2000000.0,2023-06-06T11:22:57.890000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500073,499995,"poblacion\r\nburauen,leyte\r\nburauen\r\nleyte...",not classified,2022-07-26T00:00:00,municipality of burauen,municipal corporations,asia,philippines,php,111,...,building material,51564900,2022-07-22T00:00:00,0.0,53811424,PG-Shopping- 89-2022(Ref: 8855772),NaN,Construction Materials And Supplies,141590.0,2023-06-06T11:22:53.657000
500074,499996,"airport road,\r\nlapu-lapu city\r\ncebu 6016\r...",not classified,2022-07-27T00:00:00,mactan-cebu international airport authority,air transport,asia,philippines,php,111,...,computer hardwares and consumables,51565435,2022-07-22T00:00:00,0.0,53811959,MCIAA 07-1037-22(Ref: 8858507),NaN,1 Duplex Sheet-fed Document Scanner,22495.0,2023-06-06T11:22:53.657000
500075,499997,"241050, bryansk region, g bryansk, bryansk reg...",not classified,2022-08-12T00:00:00,public procurement department of the bryansk r...,non classified,europe,russia,rub,118,...,roads,51964782,2022-08-06T00:00:00,0.0,54211077,0127200000222004543,NaN,Performance Of Contracting Work On The Object:...,80084770.0,2023-06-06T11:22:57.890000
500076,499998,karolou 1-3 athens 10437.,not classified,2022-07-25T00:00:00,ose sa,non classified,europe,greece,eur,5,...,drugs and pharmaceuticals,51540855,2022-07-21T00:00:00,0.0,53787389,NaN,NaN,Supply Of Pharmaceutical Material,0.0,2023-06-06T11:22:53.657000


In [8]:
# comments_df = df.explode("comments", ignore_index=True)
# comments_df.head(4)

,html_url,title,comments,body
0,https://github.com/huggingface/datasets/issues...,Protect master branch,"Cool, I think we can do both :)",After accidental merge commit (91c55355b634d0d...
1,https://github.com/huggingface/datasets/issues...,Protect master branch,@lhoestq now the 2 are implemented.\r\n\r\nPle...,After accidental merge commit (91c55355b634d0d...
2,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,Hi ! I guess the caching mechanism should have...,## Describe the bug\r\nAfter upgrading to data...
3,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,"If it's easy enough to implement, then yes ple...",## Describe the bug\r\nAfter upgrading to data...


In [9]:
from datasets import Dataset

comments_dataset = Dataset.from_pandas(df)
comments_dataset

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 2964
})

In [10]:
comments_dataset = comments_dataset.map(
    lambda x: {"comment_length": len(x["comments"].split())}
)

Map:   0%|          | 0/2964 [00:00<?, ? examples/s]

In [11]:
comments_dataset = comments_dataset.filter(lambda x: x["comment_length"] > 15)
comments_dataset

Filter:   0%|          | 0/2964 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length'],
    num_rows: 2175
})

In [12]:
def concatenate_text(examples):
    return {
        "text": examples["title"]
        + " \n "
        + examples["body"]
        + " \n "
        + examples["comments"]
    }


comments_dataset = comments_dataset.map(concatenate_text)

Map:   0%|          | 0/2175 [00:00<?, ? examples/s]

In [35]:
# dir(comments_dataset)

cd = comments_dataset.to_pandas()

In [40]:
cd.head(1)

,html_url,title,comments,body,comment_length,text
0,https://github.com/huggingface/datasets/issues...,Protect master branch,@lhoestq now the 2 are implemented.\r\n\r\nPle...,After accidental merge commit (91c55355b634d0d...,64,Protect master branch \n After accidental merg...


In [42]:
cd['title'][0]

'Protect master branch'

In [43]:
cd['comments'][0]

'@lhoestq now the 2 are implemented.\r\n\r\nPlease note that for the the second protection, finally I have chosen to protect the master branch only from **merge commits** (see update comment above), so no need to disable/re-enable the protection on each release (direct commits, different from merge commits, can be pushed to the remote master branch; and eventually reverted without messing up the repo history).'

In [41]:
cd['body'][0]

'After accidental merge commit (91c55355b634d0dc73350a7ddee1a6776dbbdd69) into `datasets` master branch, all commits present in the feature branch were permanently added to `datasets` master branch history, as e.g.:\r\n- 00cc036fea7c7745cfe722360036ed306796a3f2\r\n- 13ae8c98602bbad8197de3b9b425f4c78f582af1\r\n- ...\r\n\r\nI propose to protect our master branch, so that we avoid we can accidentally make this kind of mistakes in the future:\r\n- [x] For Pull Requests using GitHub, allow only squash merging, so that only a single commit per Pull Request is merged into the master branch\r\n  - Currently, simple merge commits are already disabled\r\n  - I propose to disable rebase merging as well\r\n- ~~Protect the master branch from direct pushes (to avoid accidentally pushing of merge commits)~~\r\n  - ~~This protection would reject direct pushes to master branch~~\r\n  - ~~If so, for each release (when we need to commit directly to the master branch), we should previously disable the pro

Protect master branch

In [37]:
cd['text'][0]

'Protect master branch \n After accidental merge commit (91c55355b634d0dc73350a7ddee1a6776dbbdd69) into `datasets` master branch, all commits present in the feature branch were permanently added to `datasets` master branch history, as e.g.:\r\n- 00cc036fea7c7745cfe722360036ed306796a3f2\r\n- 13ae8c98602bbad8197de3b9b425f4c78f582af1\r\n- ...\r\n\r\nI propose to protect our master branch, so that we avoid we can accidentally make this kind of mistakes in the future:\r\n- [x] For Pull Requests using GitHub, allow only squash merging, so that only a single commit per Pull Request is merged into the master branch\r\n  - Currently, simple merge commits are already disabled\r\n  - I propose to disable rebase merging as well\r\n- ~~Protect the master branch from direct pushes (to avoid accidentally pushing of merge commits)~~\r\n  - ~~This protection would reject direct pushes to master branch~~\r\n  - ~~If so, for each release (when we need to commit directly to the master branch), we should p

In [13]:
from transformers import AutoTokenizer, TFAutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = TFAutoModel.from_pretrained(model_ckpt, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFMPNetModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFMPNetModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMPNetModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFMPNetModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMPNetModel for predictions without further training.


In [14]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [15]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="tf"
    )
    encoded_input = {k: v for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [16]:
embedding = get_embeddings(comments_dataset["text"][0])
embedding.shape

TensorShape([1, 768])

In [17]:
embeddings_dataset = comments_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).numpy()[0]}
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


Map:   0%|          | 0/2175 [00:00<?, ? examples/s]

In [18]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length', 'text', 'embeddings'],
    num_rows: 2175
})

In [27]:
question = "adding features in machine learning models"
question_embedding = get_embeddings([question]).numpy()
question_embedding.shape

(1, 768)

In [28]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

In [29]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [30]:
for _, row in samples_df.iterrows():
    print(f"COMMENT: {row.comments}")
    print(f"SCORE: {row.scores}")
    print(f"TITLE: {row.title}")
    print(f"URL: {row.html_url}")
    print("=" * 50)
    print()

COMMENT: Hi
not sure how change the lable after creation, but this is an issue not dataset request. thanks 
SCORE: 39.29181671142578
TITLE: add a new column 
URL: https://github.com/huggingface/datasets/issues/1954

COMMENT: Using batched map is probably the easiest way at the moment.
What kind of augmentation would you like to do ?
SCORE: 39.269317626953125
TITLE: How to augment data ?
URL: https://github.com/huggingface/datasets/issues/365

COMMENT: This should be accessible now as a feature in dataset.info.features (and even have the mapping methods).
SCORE: 38.257083892822266
TITLE: [Feature] Keep the list of labels of a dataset as metadata
URL: https://github.com/huggingface/datasets/issues/4

COMMENT: Hi ! Currently you have to use `map` . You can see an example of how to do it in this comment: https://github.com/huggingface/datasets/issues/853#issuecomment-727872188

In the future we'll add support for a more native way of adding a new column ;)
SCORE: 37.64003372192383
TITLE: a